## **1. Dataset preperation**

**Preparing, cleaning, normalizint the data used. Clean dataset is exported as a .csv file afterwards, so this script only needs to be run once. The created file will be used in the principal component analysis.**


Taget Variable: DIABETE4: (Have you ever been told you had diabetes?)
Values: 
- 1: Yes 
- 2: only during pregnancy
- 3: No
- 4: prediabetes/borderline diabetes (so diabetes-risk?)
- 7: Dont know (how tf dont u know?
- 9: refused answer)
- BLANK: missing/not asked


drop: BLANKS, 9, 7 
combine: 2 & 3 as 'no'
keep 4 ( classify as risk)

After cleaning: 
- 0: No
- 1: Yes
- 2: Is/was at risk

In [1]:
## Imports ##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
data = pd.read_sas('./data/LLCP2021.XPT')

In [ ]:
###################################################################### Dataset preperation ###

# removing 7, 9, NaN from data
data.drop(data.loc[data["DIABETE4"]==7].index, inplace = True)
data.drop(data.loc[data["DIABETE4"]==9].index, inplace = True)
data = data.dropna(subset=["DIABETE4"])
pd.unique(data["DIABETE4"])

In [ ]:
pca_df = pd.DataFrame(data, columns = data.columns)

# remove colums containing string values (TODO: if time change some of those later to 1-0 etc...)
cleaned_dataframe_usable = pca_df.dropna(axis=1)

In [ ]:
# define DIABETE4 as target var, remove from input variables
target = pd.DataFrame(pca_df['DIABETE4'])
features = cleaned_dataframe_usable.drop(['DIABETE4'], axis = 1)

In [ ]:
# combine answers by changing "3" (no) answers to "0"(no) 
# combine answers by changing "2" (during pregnancy) answers to "0"(no) 
# combine answers by changing "4" (at risk) answers to "0"(no) 
# TODO: include at risk later on!
#target.replace(3.0, 0.0, inplace=True)
#target.replace(2.0, 0.0, inplace=True)
#target.replace(4.0, 0.0, inplace=True)

pca_df.replace(3.0, 0.0, inplace=True)
pca_df.replace(2.0, 0.0, inplace=True)
pca_df.replace(4.0, 0.0, inplace=True)

# check if target only consists of 3 values now, o-No, 1-Yes, 2-at risk
# print(pd.unique(target["DIABETE4"]))
# print(pd.unique(data["DIABETE4"]))

# save feature names in a variable
feature_names = list(features.columns)


In [ ]:
#print("No. of columns containing null values")
#print(len(cleaned_dataframe_usable.columns[ cleaned_dataframe_usable.isna().any()]))

#print("No. of columns not containing null values")
#print(len(  cleaned_dataframe_usable.columns[cleaned_dataframe_usable.notna().all()]))

#print("Total no. of columns in the dataframe")
#print(len(cleaned_dataframe_usable.columns))

In [ ]:
# cleaned_dataframe_usable.to_csv(r'.\data\clean_preprocessed_data.csv')

## **2.PCA**

In [ ]:
# Standartizing and normalizing data
x = cleaned_dataframe_usable.loc[:,feature_names].values
x = StandardScaler().fit_transform(x)
# np.mean(x), np.std(x)

## Format back into dataframe
feature_columns = ['feature_names' + str(i) for i in range (x.shape[1])]
normalized_df = pd.DataFrame(x, columns=feature_columns)

In [ ]:
#to wich number of features should be reduced? (max 79)
n_components = 5 

# actual PCA
pca_df = PCA(n_components=n_components)
principal_components = pca_df.fit_transform(x)

column_names = []
for i in range (0, n_components):
    count = str(i)
    column_names.append('PC'+count)

column_names = np.array(column_names)
print(column_names)

components_DF = pd.DataFrame(data = principal_components, columns=column_names)

In [ ]:
# Creating dataframe with percentage each component adds (output is sorted)
explained_var = pca_df.explained_variance_ratio_

explained_var_per = []
for i in range (len(explained_var)):    
    explained_var_per.append(explained_var[i] *100)

explained_var_df = pd.DataFrame(explained_var_per)

explained_var_df.columns =["% of Information"]
explained_var_df["feature_nr"] = column_names.tolist()
explained_var_df